In [3]:
import pandas as pd

from carla import OnlineCatalog, MLModelCatalog
from carla.recourse_methods import GrowingSpheres, Wachter, Dice, CCHVAE, Revise
from carla import Benchmark
from carla.evaluation.catalog import SuccessRate, Distance, Redundancy, AvgTime, ConstraintViolation, YNN

1. Dataset and model importation

In [4]:
dataset = OnlineCatalog("adult")

In [5]:
model = MLModelCatalog(dataset, "ann", backend="pytorch")

2. Generate a Counterfactual

In [12]:
dice = Dice(model, {})
factuals = dataset.df.sample(10)
cf_dice = dice.get_counterfactuals(factuals)

  0%|          | 0/10 [00:00<?, ?it/s]c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0' has dtype incompatible with 

In [19]:
factuals_decoded = dataset.inverse_transform(factuals)
cf_dice_decoded = dataset.inverse_transform(cf_dice)
factual_row = factuals_decoded.iloc[0]
cf_row = cf_dice_decoded.iloc[0]

df_pair = pd.DataFrame({
    "Factual": factual_row,
    "Counterfactual": cf_row
})

def highlight_diff(row):
    if row.iloc[-1] != row.iloc[-2]:
        return ['background-color: blue'] * len(row)
    else:
        return [''] * len(row)

styled_df = df_pair.style.apply(highlight_diff, axis=1)


styled_df


,Factual,Counterfactual
age,33.000000,33.000000
capital-gain,0.000000,49999.500000
capital-loss,0.000000,0.000000
education-num,10.000000,10.000000
fnlwgt,300497.000000,300497.000000
hours-per-week,40.000000,40.000000
income,0,nan
marital-status,Non-Married,Non-Married
native-country,US,US
occupation,Managerial-Specialist,Managerial-Specialist


3. Evaluate the generate cf: 

In [18]:
benchmark_dice = Benchmark(model, dice, factuals)
benchmark_dice.run_benchmark(measures=[Distance(model)])

  0%|          | 0/10 [00:00<?, ?it/s]c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with 

,L0_distance,L1_distance,L2_distance,Linf_distance
0,1.0,0.200000,0.040000,0.2000
1,2.0,1.331902,0.996539,0.9000
2,1.0,0.600000,0.360000,0.6000
3,2.0,0.139947,0.009830,0.0743
4,1.0,0.700000,0.490000,0.7000
5,1.0,0.300000,0.090000,0.3000
6,2.0,1.974030,1.948734,1.0000
7,2.0,0.972506,0.669758,0.8000
8,2.0,0.827564,0.357298,0.5000
9,2.0,0.800000,0.340000,0.5000


In [14]:
benchmark_dice = Benchmark(model, dice, factuals)
benchmark_dice.run_benchmark(measures=[SuccessRate(), Distance(model), YNN(model, {"y": 5, "cf_label": 1}),  Redundancy(model, {"cf_label": 1}),  ConstraintViolation(model), AvgTime(dice)])

  0%|          | 0/10 [00:00<?, ?it/s]c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '0.0' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  candidate_cfs.at[k, selected_features[k][0]] = random_instances.at[k, selected_features[k][0]]
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\dice_ml\explainer_interfaces\dice_random.py:116: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '1.0' has dtype incompatible with 

,Success_Rate,L0_distance,L1_distance,L2_distance,Linf_distance,y-Nearest-Neighbours,Redundancy,Constraint_Violation,avg_time
0,1.0,2.0,0.850261,0.422631,0.600000,0.62,1,1,1.495808
1,NaN,2.0,0.719545,0.490382,0.700000,NaN,1,2,NaN
2,NaN,2.0,2.000000,2.000000,1.000000,NaN,1,1,NaN
3,NaN,1.0,0.700000,0.490000,0.700000,NaN,1,1,NaN
4,NaN,2.0,1.300000,1.090000,1.000000,NaN,1,1,NaN
5,NaN,2.0,0.604478,0.182707,0.304478,NaN,1,1,NaN
6,NaN,2.0,1.474030,1.224704,1.000000,NaN,1,1,NaN
7,NaN,1.0,0.520063,0.270465,0.520063,NaN,0,1,NaN
8,NaN,1.0,1.000000,1.000000,1.000000,NaN,0,1,NaN
9,NaN,1.0,1.000000,1.000000,1.000000,NaN,0,1,NaN


Benchmark de plusieurs méthodes: 

In [2]:
import pandas as pd
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.recourse_methods import GrowingSpheres, Wachter, Dice, CCHVAE, Revise
import carla.evaluation.catalog as evaluation_catalog
from carla.evaluation import Benchmark

dataset = OnlineCatalog("adult")  # ou "compas", "give_me_some_credit"
mlmodel = MLModelCatalog(dataset, "ann", backend="pytorch")  # ici ANN Pytorch

config_cchvae = {
    "data_name": "adult",
    "n_search_samples": 100,
    "p_norm": 1,
    "step": 0.1,
    "max_iter": 1000,
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}

config_revise = {
    "data_name": "adult",
    "lambda": 0.5,
    "optimizer": "adam",
    "lr": 0.1,
    "max_iter": 1500,
    "target_class": [0, 1],
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "activFun": None,
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}


recourse_methods = {
    "GrowingSpheres": GrowingSpheres(mlmodel),
    #"Wachter": Wachter(mlmodel),
    "Dice": Dice(mlmodel),
    #"CCHVAE": CCHVAE(mlmodel, config_cchvae),
    #"Revise": Revise(mlmodel, dataset, config_revise),
}
n_instances = 10
factuals = dataset.df.sample(10)



all_results = []

for name, recourse_method in recourse_methods.items():
    metrics = [
    evaluation_catalog.YNN(mlmodel, {"y": 5, "cf_label": 1}),
    evaluation_catalog.Distance(mlmodel),
    evaluation_catalog.SuccessRate(),
    evaluation_catalog.Redundancy(mlmodel, {"cf_label": 1}),
    evaluation_catalog.ConstraintViolation(mlmodel),
    evaluation_catalog.AvgTime(recourse_method),  
    ]
    benchmark = Benchmark(mlmodel, recourse_method, factuals)
    result = benchmark.run_benchmark(metrics)
    result["Recourse_Method"] = name
    all_results.append(result)

results_df = pd.concat(all_results, ignore_index=True)
results_df.set_index("Recourse_Method", inplace=True)
results_df


c:\sgrondin\Explicability\CARLA\CARLA\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
c:\sgrondin\Explicability\CARLA\CARLA\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
c:\sgrondin\Explicability\CARLA\CARLA\.venv\lib\site-packages\torch\nn\modules\module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
c:\sgrondin\Explicability\CARLA\CARLA\carla\models\api\mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: 

TypeError: unsupported operand type(s) for /: 'NoneType' and 'int'

In [ ]:
import pandas as pd
from carla.data.catalog import OnlineCatalog
from carla.models.catalog import MLModelCatalog
from carla.recourse_methods import GrowingSpheres, Wachter, Dice, CCHVAE, Revise
import carla.evaluation.catalog as evaluation_catalog
from carla.evaluation import Benchmark
from carla.models.negative_instances import predict_negative_instances

dataset = OnlineCatalog("adult")  # ou "compas", "give_me_some_credit"
mlmodel = MLModelCatalog(dataset, "ann", backend="pytorch")  # ici ANN Pytorch

config_cchvae = {
    "data_name": "adult",
    "n_search_samples": 100,
    "p_norm": 1,
    "step": 0.1,
    "max_iter": 1000,
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}

config_revise = {
    "data_name": "adult",
    "lambda": 0.5,
    "optimizer": "adam",
    "lr": 0.1,
    "max_iter": 1500,
    "target_class": [0, 1],
    "binary_cat_features": True,
    "vae_params": {
        "layers": [sum(mlmodel.get_mutable_mask()), 512, 256, 8],
        "activFun": None,
        "train": True,
        "lambda_reg": 0.000001,
        "epochs": 5,
        "lr": 0.001,
        "batch_size": 32
    }
}


recourse_methods = {
    "GrowingSpheres": GrowingSpheres(mlmodel),
    "Wachter": Wachter(mlmodel),
    "Dice": Dice(mlmodel),
    "CCHVAE": CCHVAE(mlmodel, config_cchvae),
    "Revise": Revise(mlmodel, dataset, config_revise),
}
n_instances = 10
factuals = dataset.df.sample(10)

metrics = [
    evaluation_catalog.YNN(mlmodel, {"y": 5, "cf_label": 1}),
    evaluation_catalog.Distance(mlmodel),
    evaluation_catalog.SuccessRate(),
    evaluation_catalog.Redundancy(mlmodel, {"cf_label": 1}),
    evaluation_catalog.ConstraintViolation(mlmodel),
    #evaluation_catalog.AvgTime({}),  # ou avec {"time": ...} si timer dispo
]

all_results = []

for name, recourse_method in recourse_methods.items():
    benchmark = Benchmark(mlmodel, recourse_method, factuals)
    result = benchmark.run_benchmark(metrics)
    result["Recourse_Method"] = name
    all_results.append(result)

results_df = pd.concat(all_results, ignore_index=True)
results_df.set_index("Recourse_Method", inplace=True)
results_df


[INFO] Start training of Variational Autoencoder... [vae.py fit]
[INFO] [Epoch: 0/5] [objective: 76.530] [vae.py fit]
[INFO] [ELBO train: 76.53] [vae.py fit]
[INFO] [ELBO train: 87.51] [vae.py fit]
[INFO] [ELBO train: 94.33] [vae.py fit]
[INFO] [ELBO train: 101.45] [vae.py fit]
[INFO] [ELBO train: 108.32] [vae.py fit]
[INFO] ... finished training of Variational Autoencoder. [vae.py fit]
[INFO] Start training of Variational Autoencoder... [vae.py fit]
[INFO] [Epoch: 0/5] [objective: 76.148] [vae.py fit]
[INFO] [ELBO train: 76.15] [vae.py fit]
[INFO] [ELBO train: 87.03] [vae.py fit]
[INFO] [ELBO train: 94.4] [vae.py fit]
[INFO] [ELBO train: 101.56] [vae.py fit]
[INFO] [ELBO train: 108.35] [vae.py fit]
[INFO] ... finished training of Variational Autoencoder. [vae.py fit]


/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/python3.9/site-packages/torch/nn/modules/module.py:1751: UserWarning: Implicit dimension choice for softmax has been deprecated. Change the call to include dim=X as an argument.
  return self._call_impl(*args, **kwargs)
/Users/suzie/CARLA/carla/models/api/mlmodel.py:153: UserWarning: cannot re-order features for non dataframe input: <class 'numpy.ndarray'>
  warnings.warn(
/Users/suzie/CARLA/.venv/lib/pytho

KeyError: "['Distance_1', 'Distance_2', 'Distance_3', 'Distance_4'] not in index"